In [1]:
import pandas as pd
import numpy as np

folder = 'H:\\My Drive\\PROJECTS\\PSI 2022-2025\\XRF fundamentals vs. MVA'

In [12]:
preds = pd.read_csv('Z:\\data_pXRF\\olympus_17_bulk\\2018runsOlympusPredictions.csv', dtype=object)

# format to get pkey to merge on
def format_date(date):
    d = date.split('/')
    new_date = d[2][-2:] + d[1] + d[0]
    return new_date

preds['date'] = preds['Date'].apply(format_date)
preds['pkey'] = preds['date'] + '_' + preds['Reading']

key = pd.read_excel('Z:\\data_pXRF\\xlogbook backups\\xlogbook_081018.xlsx', sheet_name='Sheet1')
key = key[['spectrum_number','notes (Olympus TestID)','pellet_name']].drop_duplicates()
key.columns = ['spectrum_number','TestID','pellet_name']
key = key.dropna().reset_index(drop=True)

# format to get pkey to merge on
key['date'] = key['spectrum_number'].apply(lambda x: str(x)[:6])
key['pkey'] = key['date'] + '_' + key['TestID']

df = key.merge(preds, how='left', on='pkey')

# filter for relevant data
cols = ['spectrum_number', 'TestID', 'pellet_name','pkey','Mg', 'Mg +/-', 'Al', 'Al +/-', 
'Si', 'Si +/-', 'P', 'P +/-', 'S', 'S +/-', 'Cl', 'Cl +/-', 'Ca', 'Ca +/-', 'Ti',
 'Ti +/-', 'V', 'V +/-', 'Cr', 'Cr +/-', 'Mn', 'Mn +/-', 'Fe', 'Fe +/-',
 'Co', 'Co +/-', 'Ni', 'Ni +/-', 'Cu', 'Cu +/-', 'Zn', 'Zn +/-', 'As',
 'As +/-', 'Se', 'Se +/-', 'Rb', 'Rb +/-', 'Sr', 'Sr +/-', 'Y', 'Y +/-',
 'Zr', 'Zr +/-', 'Nb', 'Nb +/-', 'Mo', 'Mo +/-', 'Ag', 'Ag +/-', 'Cd',
 'Cd +/-', 'Sn', 'Sn +/-', 'Sb', 'Sb +/-', 'Ta', 'Ta +/-', 'W', 'W +/-',
 'Hg', 'Hg +/-', 'Pb', 'Pb +/-', 'Bi', 'Bi +/-', 'Th', 'Th +/-', 'U', 'U +/-']
df = df[cols]
# export
df.to_csv('Z:\\data_pXRF\\Olympus_instrument_predictions.csv', index=False)

pred_df = df.drop(columns='spectrum_number').drop_duplicates(ignore_index=True)
# will probably have to add this later - might be those below their LOD/Q
pred_df = pred_df.replace('ND',np.nan)
# convert to float
pred_df[cols[4:]] = pred_df[cols[4:]].astype(float)

### format predictions
All in units of wt% element

In [14]:
# elements to oxides
ox = ['SiO2','TiO2','Al2O3','Fe2O3','MgO','MnO','CaO','P2O5','SO3']
conv = pd.read_csv(folder+'\\misc\\element_oxide_conversion.csv')
ox_el_key = dict(zip(conv.oxide, conv.element))
el_conv_key = dict(zip(conv.element, conv.conversion))

for m in ox:
    elem = ox_el_key[m]
    factor = el_conv_key[elem]
    
    pred_df[m+' (wt%)'] = pred_df[elem]*factor
    pred_df[m+' (wt%) +/-'] = pred_df[elem+' +/-']*factor
    
    pred_df.drop(columns=[elem, elem+' +/-'], inplace=True)
    
# trace elements to ppm
el = ['As','Bi','Cr','Cu','Mo','Nb','Ni','Pb','Rb','Sn','Sr','Ta','Th','U','V','W','Y','Zn','Zr']
for e in el:  
    pred_df[e+' (ppm)'] = pred_df[e]*10000
    pred_df[e+' (ppm) +/-'] = pred_df[e+' +/-']*10000
    
    pred_df.drop(columns=[e, e+' +/-'], inplace=True)
    
# sort columns
to_sort = list(pred_df.columns[3:])
to_sort.sort()
pred_df = pred_df[list(pred_df.columns[:3])+to_sort]

# remove elements not being used
to_drop = [x for x in pred_df.columns[3:] if '(' not in x]
pred_df.drop(columns=to_drop, inplace=True)

# export
pred_df.to_csv(folder+'\\data\\instrument_predictions.csv', index=False)

C:\Users\ytsma22c\AppData\Local\Continuum\xraylarch\envs\auto\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
